In [1]:
import pandas as pd
import xlsxwriter
from datetime import datetime

# Nombre del archivo
file_name = f'Gestion_Reservas_Master_{datetime.now().strftime("%Y%m%d")}.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
workbook = writer.book

# ==========================================
# 1. HOJA DE CONFIGURACIÓN
# ==========================================

data_listas = {
    'Canales': ['Airbnb', 'Booking', 'Instagram', 'Directo', 'Vrbo'],
    'Alojamientos': ['Cabaña Romántica Jacuzzi', 'Chalé de Montaña', 'Apto Centro', 'Habitación Vista'], 
    'Ciudades': ['Guarne', 'Medellín', 'Rionegro', 'Bogotá', 'Cartagena'],
    'Tipo_Cliente': ['Nacional', 'Internacional'],
    # NUEVA LISTA: Medios de Pago
    'Medios_Pago': ['Plataforma Airbnb', 'Plataforma Booking', 'Transferencia Bancolombia', 'Nequi', 'Daviplata', 'Efectivo', 'Tarjeta de Crédito (Datáfono)']
}

data_params = {
    'Concepto': [
        'Comisión Booking (%)', 
        'Comisión Airbnb (Split - Anfitrión) (%)', 
        'IVA sobre Comisiones (Colombia 19%)',
        'Costo Pasarela Pagos (Directo) (%)'
    ],
    'Valor': [0.15, 0.03, 0.19, 0.04], 
    'Referencia_Nombre': ['P_Booking', 'P_Airbnb', 'P_IVA', 'P_Pasarela']
}

df_listas = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data_listas.items()]))
df_params = pd.DataFrame(data_params)

df_listas.to_excel(writer, sheet_name='Configuracion', startrow=1, startcol=0, index=False)
df_params.to_excel(writer, sheet_name='Configuracion', startrow=1, startcol=8, index=False)

ws_config = writer.sheets['Configuracion']
ws_config.write(0, 0, "LISTAS DESPLEGABLES", workbook.add_format({'bold': True}))
ws_config.write(0, 8, "PARÁMETROS FINANCIEROS", workbook.add_format({'bold': True}))

# Crear Nombres Definidos (Named Ranges)
workbook.define_name('ListaCanales', '=Configuracion!$A$2:$A$10')
workbook.define_name('ListaAlojamientos', '=Configuracion!$B$2:$B$20')
workbook.define_name('ListaCiudades', '=Configuracion!$C$2:$C$20')
workbook.define_name('ListaTiposCliente', '=Configuracion!$D$2:$D$5')
workbook.define_name('ListaMediosPago', '=Configuracion!$E$2:$E$15') # Nuevo Rango

# Definir Tasas
workbook.define_name('Tasa_Booking', '=Configuracion!$J$3')
workbook.define_name('Tasa_Airbnb', '=Configuracion!$J$4')
workbook.define_name('Tasa_IVA', '=Configuracion!$J$5')
workbook.define_name('Tasa_Pasarela', '=Configuracion!$J$6')

# Formato porcentaje
fmt_pct = workbook.add_format({'num_format': '0.0%'})
ws_config.set_column('J:J', 10, fmt_pct)
ws_config.set_column('A:E', 25)

# ==========================================
# 2. HOJA RESERVAS (Target para n8n)
# ==========================================

columns_maestro = [
    'ID_Reserva',           # Col A
    'Fecha_Reserva',        # Col B
    'Canal',                # Col C
    'Alojamiento',          # Col D
    'Ciudad',               # Col E
    'Tipo_Cliente',         # Col F
    'Medio_Pago',           # Col G (NUEVO CAMPO)
    'Nombre_Huesped',       # Col H
    'CheckIn',              # Col I
    'CheckOut',             # Col J
    'Noches',               # Col K (Calculado)
    'MONTO_BASE_ALOJAMIENTO',  # Col L (Manual/n8n)
    'TARIFA_SERVICIO_HUESPED', # Col M (Manual/n8n - Solo Airbnb)
    'TOTAL_PAGADO_POR_HUESPED', # Col N (Calculado)
    'Comision_Plataforma',      # Col O (Calculado)
    'IVA_sobre_Comision',       # Col P (Calculado)
    'Retenciones_Impuestos',    # Col Q (Manual/n8n)
    'TOTAL_DEDUCCIONES',        # Col R (Calculado)
    'PAGO_REAL_RECIBIDO',       # Col S (Calculado - Match con Banco)
    'Rentabilidad_%'            # Col T (Calculado)
]

df_maestro = pd.DataFrame(columns=columns_maestro)
df_maestro.to_excel(writer, sheet_name='Reservas', index=False, startrow=1)
ws_res = writer.sheets['Reservas']

# Estilos
fmt_money = workbook.add_format({'num_format': '$ #,##0'})
fmt_date = workbook.add_format({'num_format': 'dd/mm/yyyy'})

rows = 1000 # Preparado para 1000 reservas
col_end = xlsxwriter.utility.xl_col_to_name(len(columns_maestro)-1)

ws_res.add_table(f'A2:{col_end}{rows}', {
    'columns': [
        {'header': 'ID_Reserva'},
        {'header': 'Fecha_Reserva'},
        {'header': 'Canal'},
        {'header': 'Alojamiento'},
        {'header': 'Ciudad'},
        {'header': 'Tipo_Cliente'},
        {'header': 'Medio_Pago'}, # Nuevo Header
        {'header': 'Nombre_Huesped'},
        {'header': 'CheckIn'},
        {'header': 'CheckOut'},
        {'header': 'Noches', 'formula': '=[@CheckOut]-[@CheckIn]'},
        {'header': 'MONTO_BASE_ALOJAMIENTO'}, 
        {'header': 'TARIFA_SERVICIO_HUESPED'}, 
        {'header': 'TOTAL_PAGADO_POR_HUESPED', 
         'formula': '=SI([@Canal]="Airbnb";[@MONTO_BASE_ALOJAMIENTO]+[@TARIFA_SERVICIO_HUESPED];[@MONTO_BASE_ALOJAMIENTO])'},
        {'header': 'Comision_Plataforma', 
         'formula': '=SI([@Canal]="Booking";[@TOTAL_PAGADO_POR_HUESPED]*Tasa_Booking;SI([@Canal]="Airbnb";[@MONTO_BASE_ALOJAMIENTO]*Tasa_Airbnb;SI(O([@Canal]="Instagram";[@Canal]="Directo");[@TOTAL_PAGADO_POR_HUESPED]*Tasa_Pasarela;0)))'},
        {'header': 'IVA_sobre_Comision', 'formula': '=[@Comision_Plataforma]*Tasa_IVA'},
        {'header': 'Retenciones_Impuestos'}, 
        {'header': 'TOTAL_DEDUCCIONES', 'formula': '=SUMA([@Comision_Plataforma];[@IVA_sobre_Comision];[@Retenciones_Impuestos])'},
        {'header': 'PAGO_REAL_RECIBIDO', 
         'formula': '=[@MONTO_BASE_ALOJAMIENTO]-[@TOTAL_DEDUCCIONES]',
         'format': fmt_money},
        {'header': 'Rentabilidad_%', 'formula': '=SIERROR([@PAGO_REAL_RECIBIDO]/[@TOTAL_PAGADO_POR_HUESPED];0)', 'format': fmt_pct}
    ],
    'name': 'TablaReservas',
    'style': 'TableStyleMedium9'
})

# Validaciones
ws_res.data_validation('C3:C1000', {'validate': 'list', 'source': 'ListaCanales'})
ws_res.data_validation('D3:D1000', {'validate': 'list', 'source': 'ListaAlojamientos'})
ws_res.data_validation('E3:E1000', {'validate': 'list', 'source': 'ListaCiudades'})
ws_res.data_validation('F3:F1000', {'validate': 'list', 'source': 'ListaTiposCliente'})
ws_res.data_validation('G3:G1000', {'validate': 'list', 'source': 'ListaMediosPago'}) # Nueva validación

# Formatos Visuales
ws_res.set_column('A:A', 15)
ws_res.set_column('H:H', 25) # Nombre Huesped
ws_res.set_column('I:J', 12, fmt_date)
ws_res.set_column('L:S', 18, fmt_money)

writer.close()
print("Plantilla con campo 'Medio de Pago' generada exitosamente.")

Plantilla con campo 'Medio de Pago' generada exitosamente.


In [2]:
import pandas as pd
import xlsxwriter
from datetime import datetime

# Nombre del archivo
file_name = f'Trazabilidad_Lotes.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
workbook = writer.book

# ==========================================
# 0. ESTILOS Y COLORES
# ==========================================
COLOR_PRIMARIO = '#1C8074'
COLOR_SECUNDARIO = '#666666'
COLOR_ERROR_BG = '#FFC7CE'
COLOR_ERROR_TXT = '#9C0006'

header_fmt = workbook.add_format({
    'bold': True, 'bg_color': COLOR_PRIMARIO, 'font_color': '#FFFFFF',
    'border': 1, 'border_color': COLOR_SECUNDARIO,
    'align': 'center', 'valign': 'vcenter'
})

cell_fmt = workbook.add_format({'border': 1, 'border_color': COLOR_SECUNDARIO})

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
config_data = {
    'Lista_Productos': ['Alimento Pollo Engorde', 'Alimento Cerdo Inicio', 'Sal Mineralizada', 'Concentrado Lechero', 'Harina de Pescado'],
    'Lista_Pellets': ['Pelletizadora 1', 'Pelletizadora 2', 'Pelletizadora 3', 'Línea A', 'Línea B'],
    'Tipos_Producto': ['Granel', 'Ensaque']
}

params_data = {
    'Parámetro': ['Límite Máximo Bultos', 'Alerta Sackoff Min', 'Alerta Sackoff Max', 'Año Mínimo Permitido'],
    'Valor': [45000, -2, 2, 2026]
}

df_listas = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in config_data.items()]))
df_params = pd.DataFrame(params_data)

sheet_config = 'Configuracion'
df_listas.to_excel(writer, sheet_name=sheet_config, startrow=1, startcol=0, index=False)
df_params.to_excel(writer, sheet_name=sheet_config, startrow=1, startcol=5, index=False)

ws_config = writer.sheets[sheet_config]
ws_config.write('A1', 'LISTAS DESPLEGABLES', header_fmt)
ws_config.write('F1', 'PARÁMETROS DEL SISTEMA', header_fmt)

# Nombres Definidos
workbook.define_name('ListaProductos', f'={sheet_config}!$A$3:$A$100')
workbook.define_name('ListaPellets', f'={sheet_config}!$B$3:$B$100')
workbook.define_name('ListaTipos', f'={sheet_config}!$C$3:$C$100')
workbook.define_name('Param_MaxBultos', f'={sheet_config}!$G$3')
workbook.define_name('Param_MinSackoff', f'={sheet_config}!$G$4')
workbook.define_name('Param_MaxSackoff', f'={sheet_config}!$G$5')
workbook.define_name('Param_AnioMin', f'={sheet_config}!$G$6')

# ==========================================
# 2. TABLA PRINCIPAL
# ==========================================
sheet_name = 'Trazabilidad_Sackoff'
columns = [
    'Fecha', 'Lote', 'Producto', 'Pellet', 
    'Cant_Bultos_Planeados', 'Cant_Bultos_Producidos', 
    'Peso_Bascula', 'Diferencia', 'Sackoff', 'Observacion', 'Tipo_Producto'
]

df_main = pd.DataFrame(columns=columns)
df_main.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1, header=False)
ws_main = writer.sheets[sheet_name]

rows = 1000
col_end_char = xlsxwriter.utility.xl_col_to_name(len(columns)-1)

# Fórmulas con referencias estructuradas y control de celdas vacías
formula_diferencia = '=IF(ISBLANK([Cant_Bultos_Planeados]),"",[Cant_Bultos_Producidos]-[Cant_Bultos_Planeados])'
formula_sackoff = '=IF(OR(ISBLANK([Cant_Bultos_Planeados]), [Cant_Bultos_Planeados]=0),"",([Cant_Bultos_Producidos]-[Cant_Bultos_Planeados])/[Cant_Bultos_Planeados])'

ws_main.add_table(f'A2:{col_end_char}{rows}', {
    'columns': [
        {'header': 'Fecha', 'header_format': header_fmt},
        {'header': 'Lote', 'header_format': header_fmt},
        {'header': 'Producto', 'header_format': header_fmt},
        {'header': 'Pellet', 'header_format': header_fmt},
        {'header': 'Cant_Bultos_Planeados', 'header_format': header_fmt},
        {'header': 'Cant_Bultos_Producidos', 'header_format': header_fmt},
        {'header': 'Peso_Bascula', 'header_format': header_fmt},
        {'header': 'Diferencia', 'formula': formula_diferencia, 'header_format': header_fmt},
        {'header': 'Sackoff', 'formula': formula_sackoff, 'header_format': header_fmt},
        {'header': 'Observacion', 'header_format': header_fmt},
        {'header': 'Tipo_Producto', 'header_format': header_fmt}
    ],
    'name': 'TablaSackoff',
    'style': None 
})

# ==========================================
# 3. VALIDACIONES CON MENSAJES DE AYUDA (INPUT MESSAGE)
# ==========================================

# A. Fecha
ws_main.data_validation(f'A3:A{rows}', {
    'validate': 'custom',
    'value': '=AND(A3>=DATE(2026,1,1), A3<=TODAY())',
    'error_title': 'Fecha Inválida',
    'error_message': 'La fecha debe ser formato YYYY-MM-DD, desde 2026 y no mayor a hoy.',
    'input_title': 'Formato Fecha', 
    'input_message': 'Ingrese fecha: YYYY-MM-DD'
})

# B. Lote
ws_main.data_validation(f'B3:B{rows}', {
    'validate': 'integer', 'criteria': '>', 'value': 0,
    'error_message': 'El Lote debe ser numérico mayor a 0.'
})

# C. PRODUCTO
ws_main.data_validation(f'C3:C{rows}', {
    'validate': 'list', 'source': 'ListaProductos',
    'error_type': 'stop',
    'error_title': 'Producto Inválido',
    'error_message': 'Seleccione un valor de la lista.'
})

# D. PELLET
ws_main.data_validation(f'D3:D{rows}', {
    'validate': 'list', 'source': 'ListaPellets',
    'error_type': 'stop',
    'error_message': 'Valor inválido.'
})

# === AQUÍ ESTÁN LOS CAMBIOS PARA LOS DECIMALES ===

# E. Bultos Planeados
ws_main.data_validation(f'E3:E{rows}', {
    'validate': 'decimal', 'criteria': 'between', 
    'minimum': 0.01, 'maximum': '=Param_MaxBultos',
    'error_title': 'Error de Formato o Rango',
    'error_message': 'El valor debe ser numérico y estar dentro del rango permitido.\nRecuerde usar COMA (,) para decimales.',
    
    # Mensaje de ayuda al seleccionar la celda
    'input_title': 'Formato Numérico',
    'input_message': 'Use COMA (,) para decimales.\nNO use puntos (.).\nEjemplo: 45,5'
})

# F. Bultos Producidos
ws_main.data_validation(f'F3:F{rows}', {
    'validate': 'decimal', 'criteria': 'between', 
    'minimum': 0, 'maximum': '=Param_MaxBultos',
    'error_title': 'Error de Formato o Rango',
    'error_message': 'El valor debe ser numérico y estar dentro del rango permitido.\nRecuerde usar COMA (,) para decimales.',
    
    # Mensaje de ayuda al seleccionar la celda
    'input_title': 'Formato Numérico',
    'input_message': 'Use COMA (,) para decimales.\nNO use puntos (.).\nEjemplo: 45,5'
})

# G. Peso Báscula
ws_main.data_validation(f'G3:G{rows}', {
    'validate': 'custom',
    'value': '=ISNUMBER(G3)',
    'ignore_blank': True,
    'error_title': 'Dato No Numérico',
    'error_message': 'Este campo solo acepta números.\nSi está intentando poner decimales, asegúrese de usar COMA (,).',
    
    # Mensaje de ayuda al seleccionar la celda
    'input_title': 'Atención Decimales',
    'input_message': 'Si ingresa decimales, sepárelos con COMA (,).\nEl punto (.) puede causar error.'
})

# K. Tipo Producto
ws_main.data_validation(f'K3:K{rows}', {
    'validate': 'list', 'source': 'ListaTipos',
    'error_type': 'stop',
    'error_message': 'Valor no permitido.'
})

# ==========================================
# 4. FORMATOS VISUALES
# ==========================================
red_format = workbook.add_format({'bg_color': COLOR_ERROR_BG, 'font_color': COLOR_ERROR_TXT})

ws_main.conditional_format(f'I3:I{rows}', {
    'type': 'cell', 'criteria': '<', 'value': '=Param_MinSackoff', 'format': red_format
})
ws_main.conditional_format(f'I3:I{rows}', {
    'type': 'cell', 'criteria': '>', 'value': '=Param_MaxSackoff', 'format': red_format
})

# Bordes
ws_main.conditional_format(f'A3:{col_end_char}{rows}', {'type': 'no_blanks', 'format': cell_fmt})
ws_main.conditional_format(f'A3:{col_end_char}{rows}', {'type': 'blanks', 'format': cell_fmt})

# Anchos
ws_main.set_column('A:A', 15)
ws_main.set_column('C:C', 25)
ws_main.set_column('E:G', 18)
ws_main.set_column('H:I', 12)

writer.close()
print(f"Archivo Final '{file_name}' generado con avisos de decimales (COMA).")

Archivo Final 'Trazabilidad_Lotes.xlsx' generado con avisos de decimales (COMA).


In [6]:
import pandas as pd
import xlsxwriter
from datetime import datetime

# Nombre del archivo
file_name = f'Trazabilidad_Lotes.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
workbook = writer.book

# ==========================================
# 0. ESTILOS Y COLORES CORPORATIVOS
# ==========================================
COLOR_PRIMARIO = '#1C8074'
COLOR_SECUNDARIO = '#666666'
COLOR_ERROR_BG = '#FFC7CE'
COLOR_ERROR_TXT = '#9C0006'

# Estilos Base
header_fmt = workbook.add_format({
    'bold': True, 'bg_color': COLOR_PRIMARIO, 'font_color': '#FFFFFF',
    'border': 1, 'border_color': COLOR_SECUNDARIO,
    'align': 'center', 'valign': 'vcenter', 'text_wrap': True
})

cell_fmt = workbook.add_format({'border': 1, 'border_color': COLOR_SECUNDARIO})
time_fmt = workbook.add_format({'num_format': 'hh:mm', 'border': 1, 'border_color': COLOR_SECUNDARIO})
num_fmt  = workbook.add_format({'num_format': '#,##0.00', 'border': 1, 'border_color': COLOR_SECUNDARIO})

# ==========================================
# 1. CONFIGURACIÓN (Listas y Parámetros)
# ==========================================
config_lists = {
    'Lista_Productos': ['Alimento Pollo', 'Alimento Cerdo', 'Sal Mineral', 'Concentrado', 'Harina Pescado'],
    'Lista_Pellets':   ['Pelletizadora 1', 'Pelletizadora 2', 'Línea A'],
    'Lista_Tipos':     ['Granel', 'Ensaque'],
    'Lista_Operarios': ['Juan Perez', 'Maria Gomez', 'Carlos Ruiz', 'Operario 4'],
    'Lista_Turnos':    ['Mañana', 'Tarde', 'Noche'],
    'Lista_Paros':     ['Falla Mecánica', 'Falla Eléctrica', 'Falta Material', 'Descanso', 'Limpieza'],
    'Lista_Dados':     ['Dado 4mm', 'Dado 6mm', 'Dado 8mm'],
    'Lista_Rollers':   ['Roller A', 'Roller B', 'Roller C'],
    'Opcion_SiNo':     ['Si', 'No']
}

# Normalizar listas
max_len = max(len(v) for v in config_lists.values())
for k in config_lists:
    config_lists[k] += [''] * (max_len - len(config_lists[k]))

config_params = {
    'Parámetro': [
        'Límite Max Bultos', 'Sackoff Min', 'Sackoff Max', 'Año Mínimo',
        'Tons Min', 'Tons Max',
        'Carga Min', 'Carga Max',
        'Durabilidad Min', 'Durabilidad Max',
        'Finos Min', 'Finos Max',
        # --- NUEVOS PARÁMETROS DE PROCESO ---
        'Temp Acond Min', 'Temp Acond Max',    # 60 - 95
        'Humedad PT Min', 'Humedad PT Max',    # 8 - 13
        'Temp Enfriador Min', 'Temp Enfriador Max', # Ej: 20 - 45
        'Flujo Vapor Min', 'Flujo Vapor Max'   # Ej: 100 - 2000
    ],
    'Valor': [
        45000, -2, 2, 2026,
        0.1, 60,
        10, 200,
        80, 90,
        0, 20,
        # Valores por defecto para nuevos campos
        60, 95,     # Temp Acond
        8, 13,      # Humedad
        20, 50,     # Temp Salida Enfriador (Ajustable)
        50, 5000    # Flujo Vapor Kg/H (Ajustable)
    ]
}

df_listas = pd.DataFrame(config_lists)
df_params = pd.DataFrame(config_params)

sheet_config = 'Configuracion'
df_listas.to_excel(writer, sheet_name=sheet_config, startrow=1, startcol=0, index=False)
df_params.to_excel(writer, sheet_name=sheet_config, startrow=1, startcol=12, index=False)

ws_config = writer.sheets[sheet_config]
ws_config.write('A1', 'LISTAS MAESTRAS', header_fmt)
ws_config.write('M1', 'PARÁMETROS GLOBALES', header_fmt)

# Definir Nombres de Rangos Listas
workbook.define_name('ListaProductos', f'={sheet_config}!$A$3:$A$100')
workbook.define_name('ListaPellets',   f'={sheet_config}!$B$3:$B$100')
workbook.define_name('ListaTipos',     f'={sheet_config}!$C$3:$C$100')
workbook.define_name('ListaOperarios', f'={sheet_config}!$D$3:$D$100')
workbook.define_name('ListaTurnos',    f'={sheet_config}!$E$3:$E$100')
workbook.define_name('ListaParos',     f'={sheet_config}!$F$3:$F$100')
workbook.define_name('ListaDados',     f'={sheet_config}!$G$3:$G$100')
workbook.define_name('ListaRollers',   f'={sheet_config}!$H$3:$H$100')
workbook.define_name('ListaSiNo',      f'={sheet_config}!$I$3:$I$4')

# Parámetros numéricos (Mapeo de filas en columna N)
# N3=Bultos, N4=MinSack, N5=MaxSack, N6=Año, N7=TonsMin, N8=TonsMax, N9=CargaMin, N10=CargaMax
# N11=DuraMin, N12=DuraMax, N13=FinosMin, N14=FinosMax
workbook.define_name('Param_MaxBultos',   f'={sheet_config}!$N$3')
workbook.define_name('Param_MinSackoff',  f'={sheet_config}!$N$4')
workbook.define_name('Param_MaxSackoff',  f'={sheet_config}!$N$5')
workbook.define_name('Param_AnioMin',     f'={sheet_config}!$N$6')
workbook.define_name('Param_TonsMin',     f'={sheet_config}!$N$7')
workbook.define_name('Param_TonsMax',     f'={sheet_config}!$N$8')
workbook.define_name('Param_CargaMin',    f'={sheet_config}!$N$9')
workbook.define_name('Param_CargaMax',    f'={sheet_config}!$N$10')
workbook.define_name('Param_DuraMin',     f'={sheet_config}!$N$11')
workbook.define_name('Param_DuraMax',     f'={sheet_config}!$N$12')
workbook.define_name('Param_FinosMin',    f'={sheet_config}!$N$13')
workbook.define_name('Param_FinosMax',    f'={sheet_config}!$N$14')
# Nuevos Params
workbook.define_name('Param_TempAcondMin', f'={sheet_config}!$N$15')
workbook.define_name('Param_TempAcondMax', f'={sheet_config}!$N$16')
workbook.define_name('Param_HumPTMin',     f'={sheet_config}!$N$17')
workbook.define_name('Param_HumPTMax',     f'={sheet_config}!$N$18')
workbook.define_name('Param_TempEnfMin',   f'={sheet_config}!$N$19')
workbook.define_name('Param_TempEnfMax',   f'={sheet_config}!$N$20')
workbook.define_name('Param_VaporMin',     f'={sheet_config}!$N$21')
workbook.define_name('Param_VaporMax',     f'={sheet_config}!$N$22')


# ==============================================================================
# 2. HOJA 1: TRAZABILIDAD SACKOFF
# ==============================================================================
sheet_sack = 'Trazabilidad_Sackoff'
cols_sack = ['Fecha', 'Lote', 'Producto', 'Pellet', 'Cant_Bultos_Planeados', 'Cant_Bultos_Producidos', 
             'Peso_Bascula', 'Diferencia', 'Sackoff', 'Observacion', 'Tipo_Producto']

df_sack = pd.DataFrame(columns=cols_sack)
df_sack.to_excel(writer, sheet_name=sheet_sack, index=False, startrow=1, header=False)
ws_sack = writer.sheets[sheet_sack]
rows = 1000
end_char_sack = xlsxwriter.utility.xl_col_to_name(len(cols_sack)-1)

f_diferencia = '=IF(ISBLANK([Cant_Bultos_Planeados]),"",[Cant_Bultos_Producidos]-[Cant_Bultos_Planeados])'
f_sackoff = '=IF(OR(ISBLANK([Cant_Bultos_Planeados]),[Cant_Bultos_Planeados]=0),"",([Cant_Bultos_Producidos]-[Cant_Bultos_Planeados])/[Cant_Bultos_Planeados])'

ws_sack.add_table(f'A2:{end_char_sack}{rows}', {
    'columns': [
        {'header': 'Fecha', 'header_format': header_fmt},
        {'header': 'Lote', 'header_format': header_fmt},
        {'header': 'Producto', 'header_format': header_fmt},
        {'header': 'Pellet', 'header_format': header_fmt},
        {'header': 'Cant_Bultos_Planeados', 'header_format': header_fmt},
        {'header': 'Cant_Bultos_Producidos', 'header_format': header_fmt},
        {'header': 'Peso_Bascula', 'header_format': header_fmt},
        {'header': 'Diferencia', 'formula': f_diferencia, 'header_format': header_fmt},
        {'header': 'Sackoff', 'formula': f_sackoff, 'header_format': header_fmt},
        {'header': 'Observacion', 'header_format': header_fmt},
        {'header': 'Tipo_Producto', 'header_format': header_fmt}
    ],
    'name': 'TablaSackoff', 'style': None
})

# Validaciones Sackoff
val_fecha = {
    'validate': 'custom', 'value': '=AND(YEAR(A3)=YEAR(TODAY()), A3<=TODAY())',
    'error_title': 'Fecha Inválida', 'error_message': 'La fecha debe corresponder al año actual y no ser futura.',
    'input_title': 'Ayuda Fecha', 'input_message': 'Formato: YYYY-MM-DD\nSolo año actual.'
}
ws_sack.data_validation(f'A3:A{rows}', val_fecha)
ws_sack.data_validation(f'B3:B{rows}', {'validate': 'integer', 'criteria': '>', 'value': 0})
ws_sack.data_validation(f'C3:C{rows}', {'validate': 'list', 'source': 'ListaProductos', 'error_type': 'stop'})
ws_sack.data_validation(f'D3:D{rows}', {'validate': 'list', 'source': 'ListaPellets', 'error_type': 'stop'})
msg_decimal = {'input_title': 'Formato Numérico', 'input_message': 'Use COMA (,) para decimales.', 'error_message': 'Debe ser numérico. Use coma.'}
ws_sack.data_validation(f'E3:E{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': 0.01, 'maximum': '=Param_MaxBultos', **msg_decimal})
ws_sack.data_validation(f'F3:F{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': 0, 'maximum': '=Param_MaxBultos', **msg_decimal})
ws_sack.data_validation(f'G3:G{rows}', {'validate': 'custom', 'value': '=ISNUMBER(G3)', 'ignore_blank': True, **msg_decimal})
ws_sack.data_validation(f'K3:K{rows}', {'validate': 'list', 'source': 'ListaTipos'})
ws_sack.conditional_format(f'I3:I{rows}', {'type': 'cell', 'criteria': '<', 'value': '=Param_MinSackoff', 'format': workbook.add_format({'bg_color': COLOR_ERROR_BG})})
ws_sack.conditional_format(f'I3:I{rows}', {'type': 'cell', 'criteria': '>', 'value': '=Param_MaxSackoff', 'format': workbook.add_format({'bg_color': COLOR_ERROR_BG})})
ws_sack.set_column('A:K', 15)

# ==============================================================================
# 3. HOJA 2: TRAZABILIDAD PROCESO
# ==============================================================================
sheet_proc = 'Trazabilidad_Proceso'
cols_proc = [
    'Fecha', 'Lote', 'Producto', 'Operario', 'Turno', 'Pellet', 
    'Hora_Inicio', 'Hora_Fin', 'Toneladas_Plan', 
    'Horas_Proceso', 'Rendimiento',
    'Carga', 'Durabilidad', 'Finos',
    'Temp_Acondicionador', 'Humedad_PT', 'Temp_Salida_Enfriador', 'Flujo_Vapor_KgH', # <--- NUEVAS COLUMNAS
    'Existe_Paro', 'Tipo_Paro', 'Tiempo_Minutos_Paro', 
    'Dado', 'Roller', 'Observacion'
]

df_proc = pd.DataFrame(columns=cols_proc)
df_proc.to_excel(writer, sheet_name=sheet_proc, index=False, startrow=1, header=False)
ws_proc = writer.sheets[sheet_proc]
end_char_proc = xlsxwriter.utility.xl_col_to_name(len(cols_proc)-1)

# Fórmulas
f_horas_proceso = '''=IF(OR(ISBLANK([Hora_Inicio]),ISBLANK([Hora_Fin])),"",
    (MOD([Hora_Fin]-[Hora_Inicio],1)*24) - IF(ISNUMBER([Tiempo_Minutos_Paro]),[Tiempo_Minutos_Paro]/60,0)
)'''.replace('\n', '').replace('    ', '')

f_rendimiento = '=IF(OR(ISBLANK([Toneladas_Plan]), [Horas_Proceso]=0, [Horas_Proceso]=""), "", [Toneladas_Plan]/[Horas_Proceso])'

ws_proc.add_table(f'A2:{end_char_proc}{rows}', {
    'columns': [
        {'header': 'Fecha', 'header_format': header_fmt},
        {'header': 'Lote', 'header_format': header_fmt},
        {'header': 'Producto', 'header_format': header_fmt},
        {'header': 'Operario', 'header_format': header_fmt},
        {'header': 'Turno', 'header_format': header_fmt},
        {'header': 'Pellet', 'header_format': header_fmt},
        {'header': 'Hora_Inicio', 'header_format': header_fmt},
        {'header': 'Hora_Fin', 'header_format': header_fmt},
        {'header': 'Toneladas_Plan', 'header_format': header_fmt},
        {'header': 'Horas_Proceso', 'formula': f_horas_proceso, 'header_format': header_fmt},
        {'header': 'Rendimiento',   'formula': f_rendimiento,   'header_format': header_fmt},
        {'header': 'Carga', 'header_format': header_fmt},
        {'header': 'Durabilidad', 'header_format': header_fmt},
        {'header': 'Finos', 'header_format': header_fmt},
        # NUEVAS
        {'header': 'Temp_Acondicionador', 'header_format': header_fmt},
        {'header': 'Humedad_PT', 'header_format': header_fmt},
        {'header': 'Temp_Salida_Enfriador', 'header_format': header_fmt},
        {'header': 'Flujo_Vapor_KgH', 'header_format': header_fmt},
        
        {'header': 'Existe_Paro', 'header_format': header_fmt},
        {'header': 'Tipo_Paro', 'header_format': header_fmt},
        {'header': 'Tiempo_Minutos_Paro', 'header_format': header_fmt},
        {'header': 'Dado', 'header_format': header_fmt},
        {'header': 'Roller', 'header_format': header_fmt},
        {'header': 'Observacion', 'header_format': header_fmt}
    ],
    'name': 'TablaProceso', 'style': None
})

# --- VALIDACIONES PROCESO ---
ws_proc.data_validation(f'A3:A{rows}', val_fecha)
ws_proc.data_validation(f'B3:B{rows}', {'validate': 'integer', 'criteria': '>', 'value': 0})
ws_proc.data_validation(f'C3:C{rows}', {'validate': 'list', 'source': 'ListaProductos', 'error_type': 'stop'})
ws_proc.data_validation(f'D3:D{rows}', {'validate': 'list', 'source': 'ListaOperarios', 'error_type': 'stop'})
ws_proc.data_validation(f'E3:E{rows}', {'validate': 'list', 'source': 'ListaTurnos', 'error_type': 'stop'})
ws_proc.data_validation(f'F3:F{rows}', {'validate': 'list', 'source': 'ListaPellets', 'error_type': 'stop'})
ws_proc.data_validation(f'G3:H{rows}', {'validate': 'time', 'criteria': 'between', 'minimum': 0, 'maximum': 0.999999, 'error_message': 'Formato HH:MM inválido.'})
ws_proc.data_validation(f'I3:I{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_TonsMin', 'maximum': '=Param_TonsMax', **msg_decimal})
ws_proc.data_validation(f'L3:L{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_CargaMin', 'maximum': '=Param_CargaMax', **msg_decimal})
ws_proc.data_validation(f'M3:M{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_DuraMin', 'maximum': '=Param_DuraMax', 'error_type': 'stop', **msg_decimal})
ws_proc.data_validation(f'N3:N{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_FinosMin', 'maximum': '=Param_FinosMax', **msg_decimal})

# NUEVAS VALIDACIONES (VARIABLES PROCESO)
# Temp Acondicionador (60-95)
ws_proc.data_validation(f'O3:O{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_TempAcondMin', 'maximum': '=Param_TempAcondMax', 'error_message': 'Temp Acond fuera de rango.', **msg_decimal})

# Humedad PT (8-13%)
ws_proc.data_validation(f'P3:P{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_HumPTMin', 'maximum': '=Param_HumPTMax', 'error_message': 'Humedad fuera de rango (8-13%).', **msg_decimal})

# Temp Enfriador
ws_proc.data_validation(f'Q3:Q{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_TempEnfMin', 'maximum': '=Param_TempEnfMax', 'error_message': 'Temp Enfriador fuera de rango.', **msg_decimal})

# Flujo Vapor
ws_proc.data_validation(f'R3:R{rows}', {'validate': 'decimal', 'criteria': 'between', 'minimum': '=Param_VaporMin', 'maximum': '=Param_VaporMax', 'error_message': 'Flujo de Vapor fuera de rango.', **msg_decimal})

# Resto de columnas
ws_proc.data_validation(f'S3:S{rows}', {'validate': 'list', 'source': 'ListaSiNo', 'error_type': 'stop'})
ws_proc.data_validation(f'T3:T{rows}', {'validate': 'list', 'source': 'ListaParos'})
ws_proc.data_validation(f'U3:U{rows}', {'validate': 'custom', 'value': '=ISNUMBER(U3)', 'ignore_blank': True, **msg_decimal}) 
ws_proc.data_validation(f'V3:V{rows}', {'validate': 'list', 'source': 'ListaDados', 'error_type': 'stop'})
ws_proc.data_validation(f'W3:W{rows}', {'validate': 'list', 'source': 'ListaRollers', 'error_type': 'stop'})

# Formatos Visuales
ws_proc.set_column('G:H', 12, time_fmt) # Horas HH:MM
ws_proc.set_column('J:K', 12, num_fmt)  # Horas Calc y Rendimiento
ws_proc.set_column('O:R', 15, num_fmt)  # Nuevas variables numéricas

ws_proc.conditional_format(f'A3:{end_char_proc}{rows}', {'type': 'no_blanks', 'format': cell_fmt})
ws_proc.conditional_format(f'A3:{end_char_proc}{rows}', {'type': 'blanks', 'format': cell_fmt})
ws_proc.set_column('A:F', 15)
ws_proc.set_column('X:X', 30) # Obs

writer.close()
print(f"Archivo Final Actualizado: '{file_name}' generado exitosamente.")

Archivo Final Actualizado: 'Trazabilidad_Lotes.xlsx' generado exitosamente.
